# Session 4: CT Acquisition

## Aims of this Session
- Learn how to perform a simulated CT Acquisition
- Use reconstructed data to see how simulation paramaters affect quality
- Discover the effects of polychromatic beams on results
- Compare simulated samples to real data

In [ ]:
import k3d
from gvxrPython3 import gvxr
from gvxrPython3.utils import visualise
import numpy as np
import matplotlib.pyplot as plt
from utilities import recon_parallel, recon_widget, recon_cone
import cil
import ipywidgets as widgets

# Performing a CT Acquisiton
CT scanners are able to create 3D slices through an object by using multiple X-ray images (also known as projections, or radiographs) taken at various angles around a sample. The full details and terminology of how this reconstruction method works will be covered in tomorrow's training with CIL, for now we will treat it as a black-box function. The quality of projections directly impact the details in the final result, in this session we will take a look at what can effect our scan. *Some of the impacts on getting good scans are not obvious!*

## Two Case Studies

To better demonstrate what is going on, we will show two examples based on real-world use cases.

One use case is using an aluminium plate in a standard industrial/Lab CT machine, where we want results to measure hole diameters cut into the plate.

The second case study is looking at a parallel synchrotron beam to scan titanium fibers, and seeing how non-obvious beam properties can have a large impact on results.

---

# Aluminium Plate Case Study

In This case study you will learn:
- How to simulate a CT scan with gvxr using python
- How detector properties affect result quality

The aluminium plate phantom is a 50mmx50mm plate with cylindrical holes cut into the surface. The holes are in a total of 5 different sizes and do not cut through the entire plate. We would like to measure the diameters of these holes using a lab CT machine, but first we want to know if results are possible in the best-case scenario, hence using simulations.

In [ ]:
# Load stl and display
plot = k3d.plot()
with open("input_data/phantoms/doga-plate.stl", "rb") as model:
    plot += k3d.stl(model.read(),color=0xfdea4f)
plot.display()

## Task 1: Setup

With your new-found knowledge of gvxr, setup the experiment environment as follows:

1. Fixed detector 1m away from the source with a size of 216mmx36mm with a pixel pitch of 240um
2. The detector's lsf has been measured and stored in `input_data/lsf.npy`, display and apply it to the detector
3. A tungsten tube source with the power at 10keV
4. The plate model `input_data/phantoms/doga-plate.stl` is in mm and should be made from aluminium, lying face-up and in focus.

Notes for compatability with CIL:
- Ensure the detector's up vector is `[0, 0, 1]`
- The source position is `positive y`
- The detector position is `negative y`
- We will rotate around the `[0, 0, 1]` axis

In [ ]:
# GVXR Init
gvxr.createWindow(-1,True,"OPENGL")

In [ ]:
# Detector
gvxr.setDetectorPosition(0, -0.2, 0, "m")
gvxr.setDetectorNumberOfPixels(1216, 200)
gvxr.setDetectorPixelSize(180, 180, "um")
gvxr.setDetectorUpVector(0, 0, 1)
# gvxr.setLSF()

# Source
gvxr.setSourcePosition(0,0.8,0,"m")
gvxr.usePointSource()
# gvxr.setFocalSpot(0,0,20,0.02,"mm",3)
gvxr.addEnergyBinToSpectrum(10, "keV", 1)

# Plate
gvxr.removePolygonMeshesFromSceneGraph()
gvxr.loadMeshFile("plate","./input_data/phantoms/doga-plate.stl","mm")
gvxr.setElement("plate","Al")
gvxr.setDensity("plate",2.7,"g/cm3")
gvxr.displayScene()

In [ ]:
# Preview the environment
gvxr.displayScene()
plt.imshow(gvxr.takeScreenshot())
visualise()

## Task 2: Scanning
Now that the scene is setup, we'll take a look into reproducing a virtual scan of the plate, by taking x-ray images and rotating the sample with `gvxr.rotateScene()`

Complete the code snippets below by replacing ??? with the correct code. At the end, you'll have completed your own CT acquisition, which we'll use for reconstruction

In [ ]:
# Define the number of projections, along with the angle step
total_projections = 100
final_angle = 180
angle_step = final_angle / total_projections

In [ ]:
# Pre-create our results array with the size of our detector
projections = np.ndarray((total_projections, 200, 1216))

# Rotate our object by angle_step for every projection, and save it to the
# results array.
for i in range(0, total_projections):
    # Save current angular projection
    projections[i] = gvxr.computeXRayImage()
    # Rotate models
    gvxr.rotateScene(angle_step,0,0,1)

In [ ]:
# Don't forget, we need to use flatfield normalisation on the radiographs to get the correct attenuation values!
# Because our flatfield and darkfield are perfect, all we need to do is divide by the total energy
projections /= gvxr.getTotalEnergyWithDetectorResponse()

We have now taken a full radial scan of our target object. Using these projections we can reconstruct our data into tomographical slices, and see if it's possible to measure the size of our plate's cicles.

There are two provided methods: `reconstruct_parallel` and `reconstruct_cone`. A more in-depth look into CIL will be in tomorrow's training. For now, treat these functions as blackboxes.

In [ ]:
# Reconstruct the simulated projections with CIL.
recon = recon_cone(projections, 180/1000, final_angle,gvxr.getSourcePosition("mm"), gvxr.getDetectorPosition("mm"))
assert recon is not None

In [ ]:
# Interactive simple data display
display(recon_widget(projections, recon))

## Task 3: Tweaking
🎉 Congratulations, you've done a virtual CT scan!

Now take a look at your results and see how well you did!

With our new CT setup, we can explore what our lab scanner can do. Take a go at the little questions below and see if you can answer them!

1. Does your plate clip the edge of your scanner?
    - How can you fix this without changing the detector's size?
2. Are the holes visible in the reconstruction?
    - Try changing the color range to increase the contrast.
    - Does changing the source power make your results better?
3. What other values can you change to get a higher quality scan with the lab setup?
    - Do more projections help?
    - What can't you change with this lab setup that will help the CT scan?
4. In your opinion, would this experiment be feasable on the lab setup at all?

---

# Titanium Fibre Case Study

In this case study you will learn:
- How to automate this simulation process with json2gvxr
- An easy way to compare real results with simulated results
- Show how 'almost' pure sources will change real results

---

## Extension Task: Scanning your own model

Now that you've gotten used to using gvxr in different ways, select a model from input_data and perform a virtual scan.

Feel free to pick the simplest, or most complex multi-material models.
Multi-material models are guarenteed to provide more challenging results, and can be fun to play around with material composition.